# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('/datasets/users_behavior.csv')
display(data.head())
display(data.info())
data.duplicated().sum()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

0

In [2]:
data['calls'] = data['calls'].astype(int)

**Вывод**

Пропуски в данных отстутсвуют, как и явные дубликаты. Был изменен тип данных в столбце с количеством звонков, так как оно может быть только числом. Предсказываемое число 1 или 0, следовательно. перед нами стоит задача классификации.

## Разбейте данные на выборки

In [3]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [4]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

**Вывод**

В качестве целевого признака был выбран используемый тариф. Также выборка была раздлена на 3 категории: обучающая (60%), валидационная (20%) и тестовая (20%).

## Исследуйте модели

**Модель дерева решений**

In [5]:
for i in range(1, 11):
    model_tree = DecisionTreeClassifier(max_depth=i, random_state=12345)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    print('max_depth =', i, ':', accuracy)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


**Модель леса решений**

In [6]:
for est in range(1, 11):
    model_for = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model_for.fit(features_train, target_train) 
    accuracy = model_for.score(features_valid, target_valid) 
    print('max_est =', est, ':', accuracy)

max_est = 1 : 0.7107309486780715
max_est = 2 : 0.7636080870917574
max_est = 3 : 0.7387247278382582
max_est = 4 : 0.7713841368584758
max_est = 5 : 0.749611197511664
max_est = 6 : 0.7807153965785381
max_est = 7 : 0.7682737169517885
max_est = 8 : 0.7822706065318819
max_est = 9 : 0.7729393468118196
max_est = 10 : 0.7853810264385692


**Логистическая регрессия**

In [7]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)
predictions_valid = model_log.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid)
print(accuracy)

0.7107309486780715


**Модель деревьев и леса**

In [8]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid)
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.7558320373250389 Количество деревьев: 10 Максимальная глубина: 10


**Вывод**

Наилучшей сходимостью обладает модель дерева решений. С глубиной 7. Сходимость равна 0,789.

В модели леса деревьев наилучшей сходимостью обладает решенеие с количеством деревьев 8. Сходимость равна 0,786

Модель логистической регрессии обладает самой низкой сходимостью, она составляет 0,698.

Если объединить модель леса и решений, то наибольшей сходимостью (0,746) будет обладать модель с 10 деревьями, глубиной 10.

## Проверьте модель на тестовой выборке

**Модель дерева решений**

In [9]:
model_tree = DecisionTreeClassifier(max_depth=7, random_state=12345)
model_tree.fit(features_train, target_train)
predictions_test = model_tree.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
print(accuracy)

0.7993779160186625


**Модель леса решений** 

In [10]:
model_for = RandomForestClassifier(random_state=12345, n_estimators=8) 
model_for.fit(features_train, target_train) 
accuracy = model_for.score(features_test, target_test) 
print(accuracy)

0.7869362363919129


**Логичтическая модель**

In [11]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)
predictions_test = model_log.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
print(accuracy)

0.6842923794712286


**Деревья и лес решений**

In [12]:
model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=10)
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test) 
result = accuracy_score(target_test, predictions_test)
print(result)

0.8009331259720062


**Вывод**

В итоге если делать вывод, то следует отметить, что наибольшей сходимостью обладает модель объединения леса и дерева решений. Сходимость на тестовой выборке составляет 0.802.